In [1]:
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset4.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[1,2])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)

ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=10, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.149144
                      0.201001
                      0.289998
                      0.277260
                      0.493985
                      0.878743
                      0.168033
                      0.515332
                      0.277460
                      0.286487
 DT - feature_importance_vals
                     0.010848
                     0.055477
                     0.052878
                     0.007685
                     0.108218
                     0.149173
                     0.012846
                     0.076044
                     0.018985
                     0.037511
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the Ten

keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.020112
                     0.052563
                     0.078732
                     0.031031
                     0.072693
                     0.044511
                     0.010435
                     0.055144
                     0.067583
                     0.054779
Iteration 2


...

 XGB - feature_importance_vals
                      0.168951
                      0.239558
                      0.246474
                      0.247594
                      0.506338
                      0.912840
                      0.157778
                      0.475113
                      0.298994
                      0.293341
 DT - feature_importance_vals
                     0.011825
                     0.037607
                     0.030662
                     0.008657
                     0.087058
                     0.150453
                     0.010297
                     0.053840
                     0.026556
                     0.031689
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the Ten

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.174431
                      0.241078
                      0.313394
                      0.246865
                      0.517144
                      0.834255
                      0.172416
                      0.539587
                      0.266916
                      0.284566
 DT - feature_importance_vals
                     0.009311
                     0.044744
                     0.048284
                     0.014090
                     0.085303
                     0.156525
                     0.010104
                     0.065774
                     0.019279
                     0.031530
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the Ten

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.166038
                      0.244939
                      0.286089
                      0.261079
                      0.527737
                      0.881183
                      0.171464
                      0.513362
                      0.272168
                      0.291035
 DT - feature_importance_vals
                     0.009787
                     0.044179
                     0.060579
                     0.007900
                     0.090257
                     0.152564
                     0.015292
                     0.072226
                     0.021783
                     0.044884
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the Ten